In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import array
import gzip
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from collections import defaultdict
from scipy.spatial import distance
from sklearn.manifold import TSNE

In [ ]:
import pickle
with open('/content/drive/MyDrive/Project/anchor_p_n.pkl', 'rb') as f:
  anchor_p_n = pickle.load(f)

In [ ]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 29.4 MB/s 
     |████████████████████████████████| 182 kB 47.7 MB/s 
     |████████████████████████████████| 7.6 MB 12.3 MB/s 


In [ ]:
from transformers import AutoFeatureExtractor, TFResNetModel
feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-50")
model = TFResNetModel.from_pretrained("microsoft/resnet-50")

Downloading:   0%|          | 0.00/266 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/103M [00:00<?, ?B/s]

Some layers from the model checkpoint at microsoft/resnet-50 were not used when initializing TFResNetModel: ['classifier.1']
- This IS expected if you are initializing TFResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFResNetModel were initialized from the model checkpoint at microsoft/resnet-50.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFResNetModel for predictions without further training.


In [ ]:
import os
import cv2
compat = []
for s in anchor_p_n[1000:3000]:
    anchor = s[0]
    p = s[1]
    n = s[2]
    anchor_features = model(feature_extractor(cv2.imread(os.path.join('/content/drive/MyDrive/Project/scenes_final',anchor+'.jpg')),return_tensors='tf')['pixel_values'])
    o1 = tf.reshape(tf.reshape(anchor_features['last_hidden_state'],[2048,49])[:,-1],[1,2048])
    p_features = model(feature_extractor(cv2.imread(os.path.join('/content/drive/MyDrive/Project/products_final',p+'.jpg')),return_tensors='tf')['pixel_values'])
    o2 = tf.reshape(tf.reshape(p_features['last_hidden_state'],[2048,49])[:,-1],[1,2048])
    n_features = model(feature_extractor(cv2.imread(os.path.join('/content/drive/MyDrive/Project/products_final',n+'.jpg')),return_tensors='tf')['pixel_values'])
    o3 = tf.reshape(tf.reshape(n_features['last_hidden_state'],[2048,49])[:,-1],[1,2048])
    compat.append((o1,o2,1))
    compat.append((o1,o3,0))



In [ ]:
import pickle
with open('/content/drive/MyDrive/Project/compat.pkl','wb') as f:
    pickle.dump(compat,f)


In [ ]:
featDim = 2048 # Image feature dimensionality
styleDim = 5

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.00001)

In [ ]:
class CompatibilityModel(tf.keras.Model):
    def __init__(self, featDim, styleDim):
        super(CompatibilityModel, self).__init__()
        self.E1 = tf.Variable(tf.random.normal([featDim,styleDim],stddev=0.001))
        self.E2 = tf.Variable(tf.random.normal([featDim,styleDim],stddev=0.001))
        self.c = tf.Variable(0.0)
        
    def predict(self, x1, x2):
        s1 = tf.matmul(x1, self.E1)
        s2 = tf.matmul(x2, self.E2)
        return tf.math.sigmoid(self.c - tf.reduce_sum(tf.math.squared_difference(s1,s2)))

    def __call__(self, x1, x2, y):
        return -tf.math.log(self.predict(x1,x2)*(2*y - 1) - y + 1)

compat_model = CompatibilityModel(featDim, styleDim)

In [ ]:
def trainingStep(compat):
    with tf.GradientTape() as tape:
        (i1,i2,y) = random.choice(compat)
        # x1,x2 = X[i1],X[i2]
        objective = compat_model(i1,i2,y)
    gradients = tape.gradient(objective, compat_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, compat_model.trainable_variables))
    return objective.numpy()

In [ ]:
for i in range(50000):
    obj = trainingStep(compat)
    if (i % 5000 == 4999): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 5000, objective = 0.6458029
iteration 10000, objective = 0.7774348
iteration 15000, objective = 0.580425
iteration 20000, objective = 0.6951297
iteration 25000, objective = 0.76917857
iteration 30000, objective = 0.614755
iteration 35000, objective = 0.876713
iteration 40000, objective = 0.27914903
iteration 45000, objective = 0.743221
iteration 50000, objective = 0.6745087


In [ ]:

compat_model.save('/content/drive/MyDrive/Project/compat_model')
compat_model = tf.keras.models.load_model('/content/drive/MyDrive/Project/compat_model')


ValueError: ignored

In [ ]:
accuracy = []
for i in compat:
    x1,x2,y = i
    y_hat = compat_model.predict(x1,x2)
    if (tf.math.round(y_hat)==y):
        accuracy.append(1)
    else:
        accuracy.append(0)
acc = sum(accuracy)/len(accuracy)

In [ ]:
acc

0.58775